## Recreating Analysis of Piao Nature 2008

### Step 3: Analyze time series of all stations

Substeps

- Create a function that takes a stations co2 time series and outputs the data (actually, just use the function I already have.)

- Call the above function and save results in a dictionary, with keys being station names


In [16]:
from fun_ccg_fits import ccg_fits
from matplotlib import pyplot as plt
import pandas as pd
from copy import copy, deepcopy
import numpy as np


In [2]:
# Read in data
piao_co2_fm = pd.read_csv('../data/piao_2008/piao_co2_fm.csv')
piao_stations = pd.read_csv('../data/piao_2008/piao_stations.csv')

In [5]:

# Default parameter values as used in the ccgcrv code.
ccg_pars_def = {
    'shortterm': 80,
    'longterm': 667,
    'sampleinterval': 0,
    'numpolyterms': 3,
    'numharmonics': 4,
    'timezero': -1,
    'gap': 0,
    'use_gain_factor': False,
    'debug': False
}

# Set sampleinterval to 1
ccg_pars1 = deepcopy(ccg_pars_def)
ccg_pars1['sampleinterval'] = 1

ccg_output = dict()

for s in piao_stations['name'].unique():
    ystart = int(piao_stations.loc[piao_stations['name'] == s,'start_piao2008'])
    yend = int(piao_stations.loc[piao_stations['name'] == s,'end_piao2008'])
    sel = (piao_co2_fm['station'] == s) & (piao_co2_fm['year'].between(ystart, yend, inclusive='both'))
    input = piao_co2_fm.loc[sel, ['co2', 'dec_year']]
    out = ccg_fits(data=input, pars=ccg_pars1)
    ccg_output[s] = out


In [25]:
# Create dataframe with stations and results

# Here could remove bad years e.g. by using the Gini coefficient as in Forkel 2016

piao_ccgmeans = deepcopy(pd.DataFrame(piao_stations['name']))
piao_ccgmeans['amp_mean'] = np.nan
piao_ccgmeans['min_mean'] = np.nan
piao_ccgmeans['max_mean'] = np.nan
piao_ccgmeans['szc_mean'] = np.nan
piao_ccgmeans['azc_mean'] = np.nan

for s in piao_stations['name'].unique():
    res = ccg_output[s]
    amp_mean = res['yearly']['amp'].mean()
    min_mean = res['yearly']['min_value'].mean()
    max_mean = res['yearly']['max_value'].mean()
    szc_mean = res['yearly']['tcd_doy'].mean()
    azc_mean = res['yearly']['tcu_doy'].mean()
    piao_ccgmeans.loc[piao_ccgmeans['name']==s, 'amp_mean'] = np.round(amp_mean, 2)
    piao_ccgmeans.loc[piao_ccgmeans['name']==s, 'min_mean'] = np.round(min_mean, 2)
    piao_ccgmeans.loc[piao_ccgmeans['name']==s, 'max_mean'] = np.round(max_mean, 2)
    piao_ccgmeans.loc[piao_ccgmeans['name']==s, 'szc_mean'] = int(szc_mean)
    piao_ccgmeans.loc[piao_ccgmeans['name']==s, 'azc_mean'] = int(azc_mean)
